In [1]:
import qdrant_client
from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.indices import MultiModalVectorStoreIndex
import yaml


In [2]:
def get_document_metadata(filepath):
    with open(filepath, 'r') as file:
        content = file.read()
        _, front_matter, _ = content.split('---', 2)
        data = yaml.safe_load(front_matter)
    return data

In [3]:
# Create a local Qdrant vector store
client = qdrant_client.QdrantClient(path="qdrant_db")

In [4]:
text_store = QdrantVectorStore(
    client=client, collection_name="text_collection"
)
image_store = QdrantVectorStore(
    client=client, collection_name="image_collection"
)
storage_context = StorageContext.from_defaults(
    vector_store=text_store, image_store=image_store
)

In [ ]:
client.delete_collection(collection_name="text_collection")
client.delete_collection(collection_name="image_collection")

In [5]:
documents = SimpleDirectoryReader(input_dir="../docs", file_metadata=get_document_metadata).load_data()

In [ ]:
client.close()

In [6]:
# Create the MultiModal index
index = MultiModalVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

In [7]:
test_query = "Compare the GFA of retail space from projects in the pipeline for 4th Quarter 2023 compared to the 1st Quarter 2023"
# generate  retrieval results
retriever = index.as_retriever(similarity_top_k=3)
retrieval_results = retriever.retrieve(test_query)
from llama_index.core.response.notebook_utils import display_source_node
for res_node in retrieval_results:
    display_source_node(res_node, source_length=1000, show_source_metadata=True)

**Node ID:** 5cbc2bd8-08bd-40b8-bac9-735a2ff30762<br>**Similarity:** 0.8954476594801901<br>**Text:** Supply in the Pipeline - Retail Space

As at the end of 4th Quarter 2023, there was a total supply of about 621,000 sq m GFA of retail space from projects in the pipeline, compared with the 600,000 sq m GFA of retail space in the pipeline in the previous quarter.<br>**Metadata:** {'title': 'Release of 4th Quarter 2023 Real Estate Statistics', 'date': '2024-01-26', 'source_link': 'https://www.ura.gov.sg/Corporate/Media-Room/Media-Releases/pr24-03'}<br>